In [ ]:
import os
import pickle
from datetime import datetime
from enum import Enum, auto
from functools import cached_property
from itertools import chain
from pathlib import Path
from random import random, randint
from time import sleep
from typing import Dict, List

In [ ]:
import numpy as np
import pandas
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from numpy.random import normal
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
import pickle
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
# Five states are defined, and the five states are mutually exclusive, each person’s state is unique
class State(Enum):
    # There are two places to change the order of adjustment status: here, and the order of histories below (search for histories)
    eliminated = auto()
    susceptible = auto()
    recovered = auto()
    infected = auto()
    dead = auto()

In [ ]:
class Person(Agent):
    
    def __init__(self, unique_id: int, model: 'EbolaModel'):
        super().__init__(unique_id, model)
        self.model = model
        self.induced_infections = 0  # induced_infections is 0
        self.infected_days = 0    # infected days
        self.dead_days = 0  # dead days
        
        # Calculate the progression period of agents
        threshold = np.round(normal(self.model.progression_period_mean, self.model.progression_period_std))
        threshold = threshold if threshold >= self.model.progression_period_min else self.model.progression_period_min
        self.progression_threshold = threshold
        # Calculate the recovery days of agents，this happens on the basis of the progression period
        threshold = np.round(normal(self.model.recover_days_mean, self.model.recover_days_std))
        threshold = threshold if threshold >= self.model.recover_days_min else self.model.recover_days_min
        threshold += self.progression_threshold
        self.recover_threshold = threshold
        # Calculate the eliminated days of agents
        threshold = np.round(normal(self.model.eliminated_days_mean, self.model.eliminated_days_std))
        threshold = threshold if threshold >= self.model.eliminated_days_min else self.model.eliminated_days_min
        self.eliminated_threshold = threshold
        # Set the initial state to more accurately determine whether it is "susceptible" or "infected"(Probability, number line）
        self.state = State.susceptible if random() > model.initial_infected_rate else State.infected
        
    def step(self):
            self.check_eliminated()#check if remove the dead agent
            self.move_and_add_infected_time()  # move automaticaly
            self.check_recover()  # check if recover
            self.check_death()  # check if dead
            self.interact()  # check if interact with other agents
            
    def check_eliminated(self):
        # will not be elimated if not dead
        if self.state != State.dead:
            return
        
        # Current status: dead
        # dead days+1
        self.dead_days += 1
        # Check whether it exceeds the limit, and remove it if it exceeds
        if self.dead_days >= self.eliminated_threshold:
            self.state = State.eliminated
                    
    def move_and_add_infected_time(self):
        # if dead, do not move
        if self.state in (State.dead, State.eliminated):
            return
        
        #current status: susceptible, infected, recovered
        #step to new position(neighborhood)
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        new_position = self.random.choice(possible_steps)
        #move agents to new position
        self.model.grid.move_agent(self, new_position)
        
        # if infected, days+1
        if self.state == State.infected:
            self.infected_days += 1
            
    def check_recover(self):
        # if not infected, no need to recover
        if self.state not in (State.infected,):
            return
        
        # Current status: Infected
        # If the infection time does not reach the limit, it will not be recovered
        if self.infected_days <= self.recover_threshold:
            return
        
        # Current status: infected and exceeded the recovery limit time
        # Set the status to be recovered
        self.state = State.recovered
        
    def check_death(self):
        # if not infected, could not dead
        if self.state != State.infected:
            return
        
        # If still in progression period, could not dead
        if self.infected_days < self.progression_threshold:
            return
        
        # Current status: Infected
        # Determine whether death is based on probability
        if random() < self.model.death_rate:
            self.state = State.dead
            
    def interact(self):
        # if not infected, no dead, do not infect other agents
        if self.state not in (State.infected, State.dead):
            return
        
        # Current status: Infected or Dead
        # If infected, infect other agents
        #if self.state in (State.infected, ): # do not infected others if dead
        if self.state in (State.infected, State.dead):  # keep infecting others if dead
            neighbours: List[Person] = self.model.grid.get_neighbors(self.pos, moore=False, include_center=True)# get_neighbors, include moore and center
            neighbours = [n for n in neighbours if n is not self]#exclude itself
            [self.infect(neighbour) for neighbour in neighbours]#infect the neighbours
            
    def infect(self, other: 'Person'):
        # current status: self infected or dead, other unknown
        # if other is not susceptible, will not be infected
        if other.state != State.susceptible:
            return
        
        # current status: self infected or dead, other is susceptible
        #if self is "infected" not "dead", check progression period; If it is still in the progression period, no infection
        if self.state == State.infected and self.infected_days < self.progression_threshold:
            return
        
        # current status: self infected or dead, self not in progression period, other is susceptible
        # If the random number is greater than this probability, no infection
        if random() > self.model.transmission_probability:
            return
        
        # set other status as infected
        other.state = State.infected
        self.induced_infections += 1

In [ ]:
#Model class
class EbolaModel(Model):
    #Set all the parameters and the height, width of grid, we use multigrid in this model
    #Simplify the model
    #random_seed = 1578(not necessary for now)
    grid_height = 150
    grid_width = 150
    initial_infected_rate = 0.11
    transmission_probability = 0.44
    population = 5000
    death_rate = 0.1
    steps = 100
    output_path = 'output'
    progression_period_mean = 8
    progression_period_std = 1
    progression_period_min = 1
    recover_days_mean = 7
    recover_days_std = 1
    recover_days_min = 1
    eliminated_days_mean = 3
    eliminated_days_std = 2
    eliminated_days_min = 1
    #title = 'this is a title'

    # The first one is the background color, and the others correspond to the values in State in turn
    colors = ['white', '#0072BD', '#F6A9A9','#77AC30','#FFB740', '#B61919']
    
    #save the parameters value with each result
    def save_parameters(self):
        items = chain.from_iterable([vars(klass).items() for klass in self.__class__.mro()])
        parameters = [(k, v) for k, v in items if isinstance(v, (int, float, str))]
        parameters = [(k, v) for k, v in parameters if not k.startswith('_')]
        content = '\n'.join(f'{k} = {v}' for k, v in parameters)
        with open(self.path / 'parameters.txt', 'w', encoding='utf-8') as f:
            f.write(content)
    
    def __init__(self):
        super().__init__(Model)
        # random seed, just an experiment
        # np.random.seed(self.random_seed)
        # seed(self.random_seed)
        
        # Used for stored procedure variables
        self.susceptible_history = []
        self.infected_history = []
        self.recovered_history = []
        self.dead_history = []
        self.eliminated_history = []
        self.r0_history = []
 
        #set the grid of the model, use multigrid
        self.grid = MultiGrid(width=self.grid_height, height=self.grid_width, torus=True)
        self.schedule = RandomActivation(self)
        self.agents: Dict[int, Person] = {}
        
        #add agents into the multigrid
        for node in range(self.population):
            person = Person(node, self)
            # Group to random positions, one contains two possible elements, which are ranks
            self.grid.place_agent(person, (randint(0, self.grid_height - 1), randint(0, self.grid_width - 1)))
            self.schedule.add(person)
            self.agents[node] = person
        self.record()#record all the time
        
    @cached_property
    def path(self):
        try:
            f = __file__
        except NameError:
            f = '.'
        time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
        path = Path(f).absolute().parent / self.output_path / time
        path.exists() or os.makedirs(self.path)
        return path

    #instead if using datacollector, simply use the record(self) here to record all the data 
    def record(self):
        self.recovered_history.append(len([p for p in self.agents.values() if p.state == State.recovered]))
        self.dead_history.append(len([p for p in self.agents.values() if p.state == State.dead]))
        self.infected_history.append(len([p for p in self.agents.values() if p.state == State.infected]))
        self.susceptible_history.append(len([p for p in self.agents.values() if p.state == State.susceptible]))
        self.eliminated_history.append(len([p for p in self.agents.values() if p.state == State.eliminated]))
        self.r0_history.append(self.get_r0())#the r0 is Real-time calculated, add()here or use @property before r0(self)

    def get_r0(self):
        agents = self.agents.values()
        #If dead, not participate in the calculation of R0(dead, not infect)
        #agents = [value for value in agents if value.state != State.dead]
        agents = [value for value in agents if value.induced_infections]#if dead, participate in the calculation of R0(dead, infect too)
        infections = [agent.induced_infections for agent in agents]
        return sum(infections) / len(infections) if infections else 0 #calculate r0

    #step and record
    def step(self):
        self.schedule.step()
        self.record()
    
    #model run, draw the grid, plotting SIRD, export excel
    def run(self,draw=False):
        draw and self.save_parameters()
        draw and self.draw_state()
        draw and self.draw_count()
        for _ in range(self.steps):
            self.step()
        draw and self.draw_state()
        draw and self.draw_count()
        draw and self.draw_sir()
        draw and self.export_excel()
    
    #draw the grid(multigrid)-state of agents
    def draw_state(self, show=False):
        figure = plt.Figure(figsize=(30, 20))
        state_array = np.zeros((self.grid.width, self.grid.height), dtype=int)
        for cell_content, x, y in self.grid.coord_iter():
            cell_content: List[Person]
            states = [c.state.value for c in cell_content]
            state = max(states) if states else 0
            state_array[x][y] = state
        axes: plt.Axes = figure.add_subplot(111)
        axes.imshow(state_array, interpolation='nearest', cmap=ListedColormap(self.colors), vmin=0, vmax=5)
        labels = ['background'] + [i.name for i in State]
        patches = [Patch(color=color, label=label) for label, color in zip(labels, self.colors)]
        axes.legend(handles=patches)
        axes.set_title(self.title)
        show and figure.show()
        figure.savefig(self.path / f'state-{self.schedule.steps}.png', dpi=100)
    
    #draw the agent count in each cell of thr grid
    def draw_count(self, show=False):
        figure = plt.Figure(figsize=(30, 20))
        state_array = np.zeros((self.grid.width, self.grid.height), dtype=int)
        for cell_content, x, y in self.grid.coord_iter():
            state_array[x][y] = len(cell_content)
        axes: plt.Axes = figure.add_subplot(111)
        image = axes.imshow(state_array, interpolation='nearest')
        axes.set_title(self.title)
        figure.colorbar(image, orientation='vertical')
        show and figure.show()
        figure.savefig(self.path / f'count-{self.schedule.steps}.png', dpi=100)
    
    #plotting SIRD and r0,r0 is too small, so use r0*self population
    def draw_sir(self):
        figure: plt.Figure = plt.figure()
        ax: plt.Axes = figure.add_subplot(111)
            
        # When modifying the order of the five states, change the order here too
        histories = [
            self.eliminated_history,
            self.susceptible_history,
            self.recovered_history,
            self.infected_history,
            self.dead_history,
        ]
        config = list(zip((s.name for s in State), histories, self.colors[1:]))
        for label, values, color in config + [
            ('R0', [r * self.population for r in self.r0_history], 'purple'),
        ]:
            ax.plot(values, label=label, color=color)
        plt.xlabel('Day')
        plt.ylabel('Population')
        ax.legend()
        ax.set_title(self.title)
        figure.savefig(self.path / 'sir.png', dpi=300)
    
    #export excel
    def export_excel(self):
        df = pandas.DataFrame(data=np.array([
            self.r0_history,
            self.susceptible_history,
            self.infected_history,
            self.recovered_history,
            self.dead_history,
            self.eliminated_history
        ]).T, columns=['R0', 'susceptible', 'infected', 'recovered', 'dead','eliminated'])
        df.to_excel(self.path / 'data.xlsx')
        
    #def output_eliminated(self):
        #return self.eliminated_history[-1]

In [ ]:
def main():
    populations = [1000,3000,5000,7000,9000]
    # populations = list(range(100, 700, 200))
    repetition_times = 50
    result = []
    for p in populations:
        # for eliminated_days in np.arange(2, 8, 1):
        for eliminated_days in [2, 3, 4, 5, 6, 7]:
            for i in range(repetition_times):
                print(p, eliminated_days, i)

                class SubModel(EbolaModel):
                    death_rate = 0.1
                    recover_days_mean = 7
                    initial_infected_rate = 0.11
                    progression_period_mean = 8
                    transmission_probability = 0.44  

                    grid_width = 150
                    grid_height = 150
                    population = p
                    eliminated_days_mean = eliminated_days
                    steps = 100

                model = SubModel()
                model.run(draw=False)
                # Calculate the survival rate as the only output of the model for later analysis
                alive_rate = (model.susceptible_history[-1] + model.recovered_history[-1]) / p
                result.append((p, eliminated_days, i, alive_rate,
                               model.eliminated_history[-1],
                               model.susceptible_history[-1],
                               model.recovered_history[-1],
                               model.infected_history[-1],
                               model.dead_history[-1],
                               ))
    df = pandas.DataFrame(data=result, columns=[
        'population', 'eliminated_days', 'repetition', 'alive_rate',
        'eliminated_number', 'susceptible_number', 'recovered_number', 'infected_number', 'dead_number',
    ])
    time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    df.to_csv('population_eliminatedays.csv', index=False)
    #df.to_excel(f'output/data-{time}.xlsx', index=False)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
df = pandas.read_csv('population_eliminatedays.csv')
df

In [ ]:
pt = df.pivot_table(['alive_rate'], index=['population', 'eliminated_days'],
                    aggfunc=lambda v: sum(v) / len(v))
pt = pt.reset_index()
pt

In [ ]:
fig, ax = plt.subplots()
for key, group in pt.groupby(['population']):
    # print(group)
    ax.plot(group.eliminated_days, group.alive_rate, 'x', label=str(key))
    fig.savefig('population_0.25.png', dpi=300)
ax.legend()

In [ ]:
pt = df.pivot_table(['alive_rate'], index=['population', 'eliminated_days'],
                    aggfunc=('mean', 'std'))
pt = pt.reset_index()
pt

In [ ]:
def main():
    result = []
    for p in [5000]:
        for i in range(100): # iterate！
            print(i)
        #for eliminated_days in (3, 4, 5, 6):
            eliminated_days = 3
            class SubModel(EbolaModel):
                population = p
                eliminated_days_mean = eliminated_days

            model = SubModel()
            model.run(draw=False)
            result.append((i, #alive_rate,
                           model.eliminated_history[-1],
                           model.susceptible_history[-1],
                           model.recovered_history[-1],
                           model.infected_history[-1],
                           model.dead_history[-1],
                          ))
    df = pandas.DataFrame(data=result, columns=[
        'repetition', #'alive_rate',
        'eliminated_number', 'susceptible_number', 'recovered_number', 'infected_number', 'dead_number',
    ])
    time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    df.to_csv('5000_dead.csv', index=False)
    #df.to_excel(f'output/data-{time}.xlsx', index=False)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
df = pandas.read_csv('5000_dead.csv')
df

In [ ]:
def main():
    result = []
    for p in [5000]:
        for i in range(100): # iterate！
            print(i)
        #for eliminated_days in (3, 4, 5, 6):
            eliminated_days = 3
            class SubModel(EbolaModel):
                population = p
                eliminated_days_mean = eliminated_days

            model = SubModel()
            model.run(draw=False)
            result.append((i, #alive_rate,
                           model.eliminated_history[-1],
                           model.susceptible_history[-1],
                           model.recovered_history[-1],
                           model.infected_history[-1],
                           model.dead_history[-1],
                          ))
    df = pandas.DataFrame(data=result, columns=[
        'repetition', #'alive_rate',
        'eliminated_number', 'susceptible_number', 'recovered_number', 'infected_number', 'dead_number',
    ])
    time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    df.to_csv('.csv', index=False)
    #df.to_excel(f'output/data-{time}.xlsx', index=False)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
def main():
#     populations = [5000]
    death_rate = [0.25,0.4,0.55,0.7,0.85]
    repetition_times = 1
    result = []
    for d in death_rate:
        # for eliminated_days in np.arange(2, 8, 1):
        for eliminated_days in [2, 3, 4, 5, 6, 7]:
            for i in range(repetition_times):
                print(d, eliminated_days, i)
                    
                class SubModel(EbolaModel):
                    death_rate = d
                    recover_days_mean = 7
                    initial_infected_rate = 0.11
                    progression_period_mean = 8
                    transmission_probability = 0.44  
                    
                    grid_width = 150
                    grid_height = 150
                    population = 5000
                    eliminated_days_mean = eliminated_days
                    steps = 100
                    
                model = SubModel()
                model.run(draw=False)
                
                alive_rate = (model.susceptible_history[-1] + model.recovered_history[-1]) / 5000
                result.append((d, eliminated_days, i, alive_rate,
                               model.eliminated_history[-1],
                               model.susceptible_history[-1],
                               model.recovered_history[-1],
                               model.infected_history[-1],
                               model.dead_history[-1],
                              ))
    df = pandas.DataFrame(data=result, columns=[
        'death_rate', 'eliminated_days', 'repetition', 'alive_rate',
        'eliminated_number', 'susceptible_number', 'recovered_number', 'infected_number', 'dead_number',
    ])
    time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    df.to_csv('deathrate_elimi01.csv', index=False)
    #df.to_excel(f'output/data-{time}.xlsx', index=False)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
df = pandas.read_csv('deathrate_elimi01.csv')
df

In [ ]:
def main():
#     populations = [5000]
    death_rate = [0.05,0.1,0.15,0.2,0.25]
    repetition_times = 50
    result = []
    for d in death_rate:
        # for eliminated_days in np.arange(2, 8, 1):
        for eliminated_days in [2, 3, 4, 5, 6, 7]:
            for i in range(repetition_times):
                print(d, eliminated_days, i)
                    
                class SubModel(EbolaModel):
                    death_rate = d
                    recover_days_mean = 7
                    initial_infected_rate = 0.11
                    progression_period_mean = 8
                    transmission_probability = 0.44  
                    
                    grid_width = 150
                    grid_height = 150
                    population = 5000
                    eliminated_days_mean = eliminated_days
                    steps = 100
                    
                model = SubModel()
                model.run(draw=False)
               
                alive_rate = (model.susceptible_history[-1] + model.recovered_history[-1]) / 5000
                result.append((d, eliminated_days, i, alive_rate,
                               model.eliminated_history[-1],
                               model.susceptible_history[-1],
                               model.recovered_history[-1],
                               model.infected_history[-1],
                               model.dead_history[-1],
                              ))
    df = pandas.DataFrame(data=result, columns=[
        'death_rate', 'eliminated_days', 'repetition', 'alive_rate',
        'eliminated_number', 'susceptible_number', 'recovered_number', 'infected_number', 'dead_number',
    ])
    time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    df.to_csv('deathrate_elimi02.csv', index=False)
    #df.to_excel(f'output/data-{time}.xlsx', index=False)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
df = pandas.read_csv('deathrate_elimi02.csv')
df

In [ ]:
pt = df.pivot_table(['alive_rate'], index=['death_rate', 'eliminated_days'],
                    aggfunc=lambda v: sum(v) / len(v))
pt = pt.reset_index()
pt

In [ ]:
fig, ax = plt.subplots()
for key, group in pt.groupby(['death_rate']):
    # print(group)
    ax.plot(group.eliminated_days, group.alive_rate, 'x', label=str(key))
    
ax.legend()

In [ ]:
pt = df.pivot_table(['alive_rate'], index=['death_rate', 'eliminated_days'],
                    aggfunc=('mean', 'std'))
pt = pt.reset_index()
pt